In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import AdaBoostRegressor

In [3]:
df1 = pd.read_csv('entrenar2_st_1820.csv')
df2 = pd.read_csv('entrenar2_st_1921.csv')
df3 = pd.read_csv('entrenar2_st_2022.csv')
df4 = pd.read_csv('entrenar2_st_2123.csv')
archivos_st = [df1,df2,df3,df4]

In [4]:
df5 = pd.read_csv('entrenar_sc_1820.csv')
df6 = pd.read_csv('entrenar_sc_1921.csv')
df7 = pd.read_csv('entrenar_sc_2022.csv')
df8 = pd.read_csv('entrenar_sc_2123.csv')
archivos_sc = [df5,df6,df7,df8]

In [6]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
            colsample_bytree = 1, learning_rate = 0.5,
            max_depth = 5, alpha = 10, n_estimators = 15) 
ada_reg = AdaBoostRegressor(base_estimator=xg_reg, random_state=10, n_estimators=5,
                        learning_rate=0.2, loss='square')

In [7]:
predecir = pd.read_csv('predecir2_st_2426.csv')  
for df in archivos_st:
    X,y = df.drop(['ref_hash','segundos'],axis=1),df['segundos']
    ada_reg.fit(X,y)
preds = ada_reg.predict(predecir.drop(['ref_hash','segundos'],axis=1))
pred_st = pd.DataFrame(preds)
pred_st.columns = ['objetivo']
pred_st = pd.DataFrame({'ref_hash': predecir['ref_hash'],
                           'obj': pred_st['objetivo']})
pred_st.head()

,obj,ref_hash
0,174886.781250,69039685746313
1,143655.718750,345999128501141
2,58244.289062,360710529886978
3,144298.328125,365882020742330
4,99543.437500,416301579449694


In [8]:
ada_reg = AdaBoostRegressor(base_estimator=xg_reg, random_state=10, n_estimators=5,
                        learning_rate=0.2, loss='square')

In [9]:
predecir = pd.read_csv('predecir_sc_2426.csv')
for df in archivos_sc:
    X,y = df.drop(['ref_hash','segundos'],axis=1),df['segundos']
    ada_reg.fit(X,y)
preds = ada_reg.predict(predecir.drop(['ref_hash','segundos'],axis=1))
pred_sc = pd.DataFrame(preds)
pred_sc.columns = ['objetivo']
pred_sc = pd.DataFrame({'ref_hash': predecir['ref_hash'],
                           'obj': pred_sc['objetivo']})
pred_sc.head()

,obj,ref_hash
0,234206.687500,41863526108385
1,231809.390625,69039685746313
2,254048.312500,90072729247980
3,242161.359375,161514654074162
4,240260.468750,168103949904656


In [10]:
ids = pd.read_csv('ids.csv')
print(ids.shape)
ids.head()

(4037, 2)


,ref_hash,segundos
0,1000169251625791246,0
1,1000395625957344683,0
2,1003027494996471685,0
3,1006670001679961544,0
4,1007573308966476713,0


In [11]:
pred_st['ref_hash'] = pred_st['ref_hash'].astype('str')
pred_sc['ref_hash'] = pred_sc['ref_hash'].astype('str')
ids['ref_hash'] = ids['ref_hash'].astype('str')

In [12]:
pred_st = pred_st[pred_st['ref_hash'].isin(ids['ref_hash'])]
pred_sc = pred_sc[pred_sc['ref_hash'].isin(ids['ref_hash'])]

In [13]:
pred_st['ref_hash'] = pred_st['ref_hash']+'_st'
print(pred_st.shape)
pred_st.head()

(4037, 2)


,obj,ref_hash
28,86825.523438,1037575899869926_st
154,66678.843750,4700892528469466_st
168,154409.171875,4920567832392301_st
380,173481.125000,11222455493472018_st
441,111969.687500,12710129175666777_st


In [14]:
pred_sc['ref_hash'] = pred_sc['ref_hash']+'_sc'
print(pred_sc.shape)
pred_sc.head()

(4037, 2)


,obj,ref_hash
23,247854.750000,1037575899869926_sc
105,232149.125000,4920567832392301_sc
176,245544.562500,8624897563799496_sc
258,253795.500000,12710129175666777_sc
261,241172.234375,12880076216702587_sc


In [15]:
prediccion = pred_st.append(pred_sc)
prediccion = prediccion[['ref_hash','obj']]
prediccion = prediccion.sort_values('ref_hash')
print(prediccion.shape)
prediccion.head()

(8074, 2)


,ref_hash,obj
21334,1000169251625791246_sc,239376.750000
35649,1000169251625791246_st,49985.492188
247885,1000395625957344683_sc,132408.484375
35656,1000395625957344683_st,48859.425781
21393,1003027494996471685_sc,237750.078125


In [16]:
prediccion.to_csv('target_adaBoost_xgb.csv', index=False)